In [1]:
import numpy as np
import pandas as pd
import pystan

%matplotlib inline

In [2]:
def waic(log_likelihood):
    training_error = - np.mean(np.log(np.mean(np.exp(log_likelihood), axis=0)))
    functional_variance_div_N = np.mean((np.mean(log_likelihood**2, axis=0)) - np.mean(log_likelihood)**2)
    return training_error + functional_variance_div_N

In [3]:
sample = np.array(pd.read_csv("./data.csv")['value'])

In [4]:
pareto = """
data {
    int<lower=0> N;
    real<lower=1.0> x[N];
    real<lower=0> minx;
}
parameters {
    real<lower=0.0001,upper=minx>   alpha;
    real<lower=0.0001>  beta; 
}
model {
    for(n in 1:N){
        x[n] ~ pareto(alpha, beta);
    }
}
generated quantities{
    vector[N] log_lik;

    for(n in 1:N){
        log_lik[n] = pareto_lpdf(x[n]|alpha,beta);
    }
}
"""

In [5]:
log_normal = """
data {
  int<lower=0> N;
  real<lower=0.999> x[N];
}
parameters {
  real  mu;  
  real<lower=0>   sigma;
}
model {
  for(n in 1:N){
    x[n] ~ lognormal(mu,sigma);
  }
}
generated quantities{
  vector[N] log_lik;
  
  for(i in 1:N){
    log_lik[i] = lognormal_lpdf(x[i]|mu,sigma);
  }
}
"""

In [6]:
gamma = """
data {
  int<lower=0> N;
  real<lower=0.999> x[N];
}
parameters {
  real<lower=0.0001>  alpha;
  real<lower=0.0001>   beta;
}
model {
  for(i in 1:N){
    x[i] ~ gamma(alpha,beta);
  }
}
generated quantities{
  vector[N] log_lik;

  for(i in 1:N){
    log_lik[i] = gamma_lpdf(x[i]|alpha,beta);
  }
}

"""

In [7]:
data = {
    'N': len(sample),
    'x':  sample,
    'minx': 1
}

sm1 = pystan.stan(model_code=pareto, data=data)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_740310d1f50ee8aeddf083a6b02f0131 NOW.


In [8]:
data = {
    'N': len(sample),
    'x':  sample
}

sm2 = pystan.stan(model_code=log_normal, data=data)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_bc7abf8948cc9690adf089d1d95cf946 NOW.


In [9]:
data = {
    'N': len(sample),
    'x':  sample
}

sm3 = pystan.stan(model_code=gamma, data=data)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_d508aef499f08141f76ec0bc4da257e8 NOW.


In [10]:
print "pareto: ", waic(sm1.extract()['log_lik'])
print "log_normal: ", waic(sm2.extract()['log_lik'])
print "gamma", waic(sm3.extract()['log_lik'])

pareto:  5.11359657333
log_normal:  5.54704191093
gamma 6.75763087054
